# spock
Scientific-paper stats

Installing dependencies which are:
- json
- scholarly

In [1]:
import json
from scholarly import scholarly

# The Publication Class: 


In [2]:

class Publication:
    def __init__(self,publication_filled,author_name) -> None:
        self.publication_filled = publication_filled
        self.author_name = author_name
        
        self.title = self.get_publication_title()
        self.abstract = self.get_publication_abstract()
        self.author = self.get_author_name()
        self.year = self.get_year()
        self.topic = self.get_topic() 
        
    def get_topic(self,output_file="json/ouput.json",
                  input_file="json/response.json"):
        try:
            with open(output_file,'r') as file:
                data = json.load(file)
            return data[self.author]['topic']
        except Exception as e:
            return self.__get_topic(input_file)
    
    def get_publication_title(self):
        return self.publication_filled['bib']['title'] 

    def get_publication_abstract(self):
        return self.publication_filled['bib']['abstract']

    def get_author_name(self):
        return self.publication_filled['bib']['author']

    def get_year(self):
        return self.publication_filled['bib']['pub_year']
    
    def __get_topic(self, file):
        
        topics = []
        with open(file, 'r') as file:
            data = json.load(file)
        
        for category, item in data.items():
            for keyword in item['keywords']:
                if keyword in self.abstract:
                    topics.append(category)
                if keyword in self.title:
                    topics.append(category)
    
                    
        return list(set(topics))
    
    
            
        
    
        



# The Author class

In [3]:

class Author:
    def __init__(self,author) -> None:
        self.author_name = author
        self.last_publication = Publication(self.get_last_publication(),self.author_name)



    def get_last_publication(self):
        search_query = scholarly.search_author(self.author_name)
        first_author_result = next(search_query)
        author = scholarly.fill(first_author_result )
        first_publication = sorted(author['publications'], 
                                   key= lambda x: int(x['bib']['pub_year']) if 'pub_year' in x['bib'] else 0, 
                                   reverse=True)[0]
        first_publication_filled = scholarly.fill(first_publication)
        return first_publication_filled


    def setup_author(self,output_file):
        with open(output_file,'r') as file:
            data = json.load(file)
        author_last_publication = self.last_publication
        
        data[self.author_name] = {"title": author_last_publication.title,
                                    "abstract": author_last_publication.abstract,
                                    "topic": author_last_publication.topic, 
                                    "author": author_last_publication.author, 
                                    "year": author_last_publication.year}
        
        with open(output_file,'w') as file:
            json.dump(data, file)


# Code for initial set-up
- Retrieving data for the latest article for the AC members from scholarly
- Writing the data in a json file

In [ ]:
with open("authors.txt","r") as file:
    authors = file.readlines()
    for author in authors:
        try:
            author = author[:-1]
            author_filled = Author(author)
            author_filled.setup_author('json/ouput.json')
            print(f"Topics for {author} have been updated")
        except Exception as e:
            print(f"Couldn't find the google scholar profile for {author}: {e}")
